##### Library

In [ ]:
import pandas as pd
import numpy as np
import gzip
import json

##### Function

In [ ]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

##### Configuration

In [ ]:
path_data_external = '../../data/external'
path_data_raw = '../../data/raw'

file_name_products = 'products.json.gz'
file_name_reviews = 'reviews.json.gz'

##### Products

In [ ]:
df_products_external = getDF(f'{path_data_external}/{file_name_products}')

In [ ]:
df_products_external.info()

In [ ]:
df_products = df_products_external[
    ["asin", "category", "title", "description", "price", "imageURLHighRes"]
].replace("", np.NAN)

df_products = df_products.rename(columns = {
    "asin" : "product_id",
    "title" : "name",
    "imageURLHighRes" : "image_url"
})

df_products["category"] = [','.join(map(str, l)) for l in df_products['category']]
df_products = df_products[~df_products["category"].str.contains("Parts|Accessories", regex=True)]

# Data transformation (Correct data format)
df_products["description"] = df_products["description"].str.get(0).replace("", np.NAN)
df_products["price"] = df_products["price"].str.extract(r"(\d+\.\d+)")

# Data cleaning (Drop Nan value columns)
df_products = df_products.dropna(subset=["product_id", "name", "description", "price"])
df_products = df_products[df_products["image_url"].str.len() != 0]

# Data cleaning (Drop duplicate rows)
df_products = df_products.drop_duplicates(subset=["product_id"])
df_products = df_products.drop_duplicates(subset=["name"])

# Data cleaning (Remove Html)
df_products = df_products[~df_products["name"].str.contains("span|class=", regex=True)]

# Data cleaning (limit string length)
df_products = df_products[(df_products["name"].str.len() >= 5) & (df_products["name"].str.len() <= 200)]
df_products = df_products[(df_products["description"].str.len() >= 5) & (df_products["description"].str.len() <= 2000)]

# Ordering
df_products = df_products.sort_values(by=["product_id"])

In [ ]:
df_products.info()

In [ ]:
df_products.to_json(f"{path_data_raw}/{file_name_products}", compression="gzip", orient="records")

##### Reviews

In [ ]:
df_products = pd.read_json(f'{path_data_raw}/{file_name_products}', orient="records", compression="gzip")

In [ ]:
df_reviews_external = getDF(f'{path_data_raw}/{file_name_reviews}')

In [ ]:
df_reviews_external.info()

In [ ]:
# Select only useful fields
df_reviews = df_reviews_external[
    ["reviewerID", "reviewerName", "asin", "overall", "reviewText", "summary", "unixReviewTime"]
]

# Rename columns
df_reviews = df_reviews.rename(columns = {
    "reviewerID" : "user_id",
    "reviewerName" : "username",
    "asin" : "product_id",
    "overall" : "ratings",
    "reviewText" : "review_text",
    "unixReviewTime": "created_at"
})

# Drop null value columns
df_reviews = df_reviews.dropna(subset=["user_id", "username", "product_id", "ratings"])

# Drop duplicate rows
df_reviews = df_reviews.drop_duplicates(subset=["user_id", "product_id"])

# Data cleaning (limit string length)
df_reviews = df_reviews[(df_reviews["review_text"].str.len() >= 5) & (df_reviews["review_text"].str.len() <= 2000)]

# Convert created_at object to datetime
df_reviews["created_at"] = pd.to_datetime(df_reviews["created_at"], unit = "s")

In [ ]:
df_reviews.info()

In [ ]:
# Join the products table to remove unused reviews
df_reviews = pd.merge(df_products, df_reviews, how="inner", on="product_id", validate="one_to_many").sort_values(by=["product_id"])
df_reviews = df_reviews.dropna()

# Remove products table's attributes
df_reviews = df_reviews[
    ["user_id", "username", "product_id", "ratings", "review_text", "summary", "created_at"]
]

In [ ]:
df_reviews.info()

In [ ]:
df_reviews.to_json(f"{path_data_raw}/{file_name_reviews}", compression="gzip", orient="records")